In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"

using Pkg
Pkg.activate(".")

using Printf
using AtomsCalculators
using ASEconvert # use this PR:https://github.com/mfherbst/ASEconvert.jl/pull/17, Pkg.add(url="https://github.com/tjjarvinen/ASEconvert.jl.git", rev="atomscalculators")
using Unitful: Å, nm
using PythonCall
ENV["PYTHON"] = "/SNS/users/ccu/miniconda3/envs/analysis/bin/python"
# install the following packages in julia REPL
# using CondaPkg
# CondaPkg.add_pip("IPython")
# CondaPkg.add_pip("nglview")
using StaticArrays: SVector

using GLMakie
using Molly
using Zygote
using LinearAlgebra
import Interpolations:cubic_spline_interpolation, linear_interpolation, interpolate, BSpline, Cubic, scale, Line, OnGrid, extrapolate, Gridded
using DelimitedFiles

  Activating new project at `~/Documents/ABCD_J/EAM`


In [2]:
function repeat(fun,times)
    for i in 1:times
        fun()
    end
end

repeat (generic function with 1 method)

## ASE EAM for reference

In [3]:
## 1. Import ASE and other Python modules
# Import ASE and other Python modules as needed
ase = pyimport("ase")
ase_view = pyimport("ase.visualize")
ase_plot = pyimport("ase.visualize.plot")
plt = pyimport("matplotlib.pyplot")

# Import Al EAM potential
fname = "Al99.eam.alloy"
EAM_ASE = pyimport("ase.calculators.eam") # python ASE-EAM calculator
eam_cal = ASEconvert.ASEcalculator(EAM_ASE.EAM(potential=fname))  # EAM calculater, converted to AtomsBase format

## 2. Define customized interaction type in AtomsCalculators
### 2.1 Define interaction
# AtomsCalculators class containing calculator and system
struct EAMInteraction
    calculator::Any  # Holds the ASE EAM calculator reference
    atoms_ab::Any    # Holds atoms representation compatible with ASE
end

### 2.2 Customized convert_ase function for evaluating potential and interactions using the ASE EAM interaction
# Customized convert_ase function converting Molly system to ASE format: handling with charges
using UnitfulAtomic
import PeriodicTable
const uVelocity = sqrt(u"eV" / u"u")
function convert_ase_custom(system::AbstractSystem{D}) where {D}
    # print("called by Molly")
    D != 3 && @warn "1D and 2D systems not yet fully supported."

    n_atoms = length(system)
    pbc     = map(isequal(Periodic()), boundary_conditions(system))
    numbers = atomic_number(system)
    masses  = ustrip.(u"u", atomic_mass(system))

    symbols_match = [
        PeriodicTable.elements[atnum].symbol == string(atomic_symbol(system, i))
        for (i, atnum) in enumerate(numbers)
    ]
    if !all(symbols_match)
        @warn("Mismatch between atomic numbers and atomic symbols, which is not " *
              "supported in ASE. Atomic numbers take preference.")
    end

    cell = zeros(3, 3)
    for (i, v) in enumerate(bounding_box(system))
        cell[i, 1:D] = ustrip.(u"Å", v)
    end

    positions = zeros(n_atoms, 3)
    for at = 1:n_atoms
        positions[at, 1:D] = ustrip.(u"Å", position(system, at))
    end

    velocities = nothing
    if !ismissing(velocity(system))
        velocities = zeros(n_atoms, 3)
        for at = 1:n_atoms
            velocities[at, 1:D] = ustrip.(uVelocity, velocity(system, at))
        end
    end

    # We don't map any extra atom properties, which are not available in ASE as this
    # only causes a mess: ASE could do something to the atoms, but not taking
    # care of the extra properties, thus rendering the extra properties invalid
    # without the user noticing.
    charges = nothing
    magmoms = nothing
    for key in atomkeys(system)
        if key in (:position, :velocity, :atomic_symbol, :atomic_number, :atomic_mass)
            continue  # Already dealt with
        elseif key == :charge
            charges = charge.(system.atoms) #### Using the charge() function in Molly!
        elseif key == :magnetic_moment
            magmoms = system[:, :magnetic_moment]
        else
            @warn "Skipping atomic property $key, which is not supported in ASE."
        end
    end

    # Map extra system properties
    info = Dict{String, Any}()
    for (k, v) in pairs(system)
        if k in (:bounding_box, :boundary_conditions)
            continue
        elseif k in (:charge, )
            info[string(k)] = ustrip(u"e_au", v)
        elseif v isa Quantity || (v isa AbstractArray && eltype(v) <: Quantity)
            # @warn("Unitful quantities are not yet supported in convert_ase. " *
            #       "Ignoring key $k")
        else
            info[string(k)] = v
        end
    end

    ase.Atoms(; positions, numbers, masses, magmoms, charges,
              cell, pbc, velocities, info)
end

### 2.3 Force calculation
# Define customized AtomsCalculators here
function AtomsCalculators.potential_energy(system::Molly.System, interaction::EAMInteraction; kwargs...)
    # Convert Molly's system to ASE's Atoms format
    ase_atoms = convert_ase_custom(system)
    
    # Calculate potential energy using ASE's EAM calculator
    # energy = AtomsCalculators.potential_energy(ase_atoms, interaction.calculator)
    energy_py = interaction.calculator.ase_python_calculator.get_potential_energy(ase_atoms)
    energy = pyconvert(Float64, energy_py)*u"eV" # also consider unit conversion

    return energy
end

function AtomsCalculators.forces(system::Molly.System, interaction::EAMInteraction; kwargs...)
    # Convert Molly's system to ASE's Atoms format
    ase_atoms = convert_ase_custom(system)

    # Use ASE to calculate forces
    f = interaction.calculator.ase_python_calculator.get_forces(ase_atoms)

    # Reshape and rearrange into the jupyter SVector format
    tmp = pyconvert(Array{Float64}, f)
    vector_svector = [SVector{3}(tmp[i, j] for j in 1:3) for i in 1:size(tmp, 1)]
    FT = AtomsCalculators.promote_force_type(system, interaction.calculator.ase_python_calculator)
    tmp2 = [SVector{3}(tmp[i, j] for j in 1:3) for i in 1:size(tmp, 1)]
    tmp3 = reinterpret(FT, tmp2)

    return tmp3
end

## Define a Molly system wo interaction 

In [4]:
## 1. Import ASE and other Python modules
# Import ASE and other Python modules as needed
ase = pyimport("ase")

al_LatConst = 4.0495
atom_mass = 26.9815u"u"  # Atomic mass of aluminum in grams per mole
function system_adatom(size)

    # Build an (001) Al surface  
    atoms_ase = ase.build.fcc100("Al", size=size, vacuum = al_LatConst*4)
    # The basis vectors on x and y are along 1/2<110> directions
    ase.build.add_adsorbate(atoms_ase, "Al", al_LatConst/2, position=(al_LatConst*(2.5*sqrt(1/2)/2),al_LatConst*(2.5*sqrt(1/2)/2)))
    # ase.build.add_adsorbate(atoms_ase, "Al", al_LatConst/2, "bridge")

    atoms_ase.translate([al_LatConst*(sqrt(1/2)/4),al_LatConst*(sqrt(1/2)/4),0])
    atoms_ase.wrap()

    atoms_ase_cell = atoms_ase.get_cell()
    box_size = pyconvert(Array{Float64}, [atoms_ase_cell[x,x] for x in range(0,2)])/10*u"nm"

    # Build an Julia AtomsBase abstract 
    atoms_ab = pyconvert(AbstractSystem, atoms_ase)

    ## 4. Create Molly system
    ### 4.1 Convert atom positions to Molly's expected format (nanometers) and create Molly.Atom objects
    # Get atom positions from previously defined ASE system
    function get_positions(atoms_ase)
        positions = [(atom.position[1], atom.position[2], atom.position[3]) for atom in atoms_ase]
        return positions
    end

    # Convert each position from Ångströms to nanometers and ensure the conversion is applied element-wise.
    atom_positions = [SVector(uconvert(nm, pos[1]), 
        uconvert(nm, pos[2]), uconvert(nm, pos[3])) for pos in get_positions(atoms_ab)]

    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                            #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                            ) for i in 1:length(atom_positions)]
    return molly_atoms, atoms_ab, box_size, atom_positions
end

molly_atoms, atoms_ab, box_size, atom_positions = system_adatom((7,7,6))


# Specify boundary condition
boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

# Create the Molly System with atoms, positions, velocities, and boundary
molly_system = Molly.System(
    atoms=molly_atoms,
    atoms_data = [AtomData(element="Al") for a in molly_atoms],
    coords=atom_positions,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    neighbor_finder = DistanceNeighborFinder(
    eligible=trues(length(molly_atoms), length(molly_atoms)),
    n_steps=10,
    dist_cutoff=6.3/10*u"nm"),
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/nm"  # Ensure these units are correctly specified
    )

atom_positions_init = copy(atom_positions)

295-element Vector{SVector{3, Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}}:
 [0.214774845950649 nm, 0.214774845950649 nm, 1.6198000000000001 nm]
 [0.5011530923312008 nm, 0.214774845950649 nm, 1.6198000000000001 nm]
 [0.7875313387117525 nm, 0.214774845950649 nm, 1.6198000000000001 nm]
 [1.0739095850923044 nm, 0.214774845950649 nm, 1.6198000000000001 nm]
 [1.3602878314728561 nm, 0.214774845950649 nm, 1.6198000000000001 nm]
 [1.646666077853408 nm, 0.214774845950649 nm, 1.6198000000000001 nm]
 [1.9330443242339594 nm, 0.214774845950649 nm, 1.6198000000000001 nm]
 [0.214774845950649 nm, 0.5011530923312008 nm, 1.6198000000000001 nm]
 [0.5011530923312008 nm, 0.5011530923312008 nm, 1.6198000000000001 nm]
 [0.7875313387117525 nm, 0.5011530923312008 nm, 1.6198000000000001 nm]
 ⋮
 [1.789855201043684 nm, 1.5034769546631321 nm, 2.6323 nm]
 [0.07158572276037312 nm, 1.789855201043684 nm, 2.6323 nm]
 [0.3579639691409249 nm, 1.789855201043684 nm, 2.6323 nm]
 [0.6443422155214766 nm, 1.789

## Define interaction

In [5]:
# Helper function to get the type of a Interpolation object
constructor_interp = cubic_spline_interpolation

function get_spline_type(constructor_interp=cubic_spline_interpolation)
    dummy_spline = constructor_interp(0.0:1.0:2.0, [0.0, 1.0, 2.0])
    return typeof(dummy_spline)
end

# Helper function to get the type of the derivative of a CubicSplineInterpolation object
function get_deriv_type(constructor_interp=cubic_spline_interpolation)
    dummy_spline = constructor_interp(0.0:1.0:2.0, [0.0, 1.0, 2.0])
    dummy_deriv = deriv(dummy_spline)
    return typeof(dummy_deriv)
end

function deriv(spline::get_spline_type(constructor_interp))
    d_spline(x) = gradient(spline, x)[1]
    return d_spline
end

mutable struct EAM
    Nelements::Int
    elements::Vector{String}
    nrho::Int
    drho::Float64
    nr::Int
    dr::Float64
    cutoff::Float64
    embedded_data::Matrix{Float64}
    density_data::Matrix{Float64}
    Z::Vector{Int}
    mass::Vector{Float64}
    a::Vector{Float64}
    lattice::String
    rphi_data::Array{Float64, 3}
    r_range::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}
    rho_range::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}
    r::Vector{Float64}
    rho::Vector{Float64}

    embedded_energy::Vector{get_spline_type(constructor_interp)}
    electron_density::Vector{get_spline_type(constructor_interp)}
    d_embedded_energy::Vector{get_spline_type(constructor_interp)}
    d_electron_density::Vector{get_spline_type(constructor_interp)}
    phi::Matrix{get_spline_type(constructor_interp)}
    d_phi::Matrix{get_spline_type(constructor_interp)}

    EAM() = new()
end

In [6]:
function set_splines(calculator::EAM)
    calculator.embedded_energy = Vector{get_spline_type(constructor_interp)}(undef, calculator.Nelements)
    calculator.electron_density = Vector{get_spline_type(constructor_interp)}(undef, calculator.Nelements)
    calculator.d_embedded_energy = Vector{get_spline_type(constructor_interp)}(undef, calculator.Nelements)
    calculator.d_electron_density = Vector{get_spline_type(constructor_interp)}(undef, calculator.Nelements)

    for i in 1:calculator.Nelements
        calculator.embedded_energy[i] = constructor_interp(calculator.rho_range, calculator.embedded_data[i, :]) # arrays of embedded energy functions, [N_types]
        calculator.electron_density[i] = constructor_interp(calculator.r_range, calculator.density_data[i, :]) # arrays of electron density functions, [N_types]
        f_embedded_energy = deriv(calculator.embedded_energy[i])
        f_electron_density = deriv(calculator.electron_density[i])
        d_embedded_energy_knot = f_embedded_energy.(calculator.rho)
        d_electron_density_knot = f_electron_density.(calculator.r)
        calculator.d_embedded_energy[i] = constructor_interp(calculator.rho_range,d_embedded_energy_knot) # arrays of derivative of embedded energy functions, [N_types]
        calculator.d_electron_density[i] = constructor_interp(calculator.r_range,d_electron_density_knot) # arrays of derivative of electron density functions, [N_types]
        # calculator.d_embedded_energy[i] = deriv(calculator.embedded_energy[i]) # arrays of derivative of embedded energy functions, [N_types]
        # calculator.d_electron_density[i] = deriv(calculator.electron_density[i]) # arrays of derivative of electron density functions, [N_types]
    end

    calculator.phi = Matrix{get_spline_type(constructor_interp)}(undef, calculator.Nelements, calculator.Nelements) # arrays of pairwise energy functions, [N_types, N_types]
    calculator.d_phi = Matrix{get_spline_type(constructor_interp)}(undef, calculator.Nelements, calculator.Nelements) # arrays, [N_types, N_types]

    # ignore the first point of the phi data because it is forced
    # to go through zero due to the r*phi format in alloy and adp
    for i in 1:calculator.Nelements
        for j in i:calculator.Nelements
            calculator.phi[i, j] = constructor_interp(calculator.r_range[2:end], calculator.rphi_data[i, j, :][2:end] ./ calculator.r[2:end]) 
            f_d_phi = deriv(calculator.phi[i, j])
            d_phi_knot = f_d_phi.(calculator.r[2:end])
            calculator.d_phi[i, j] = constructor_interp(calculator.r_range[2:end],d_phi_knot)
            # calculator.d_phi[i, j] = deriv(calculator.phi[i, j])

            if j != i
                calculator.phi[j, i] = calculator.phi[i, j]
                calculator.d_phi[j, i] = calculator.d_phi[i, j]
            end
        end
    end
end


"""
    read_potential!(calculator::EAM, fd::String)

Reads the potential data from a file and populates the fields of the `calculator` object.

# Arguments
- `calculator::EAM`: The EAM calculator object to populate with potential data.
- `fd::String`: The file path to the potential data file.

# Description
This function reads the potential data from the specified file and assigns the values to the corresponding fields of the `calculator` object. The file should be in a specific format, with each line containing the relevant data for a specific field.

The function starts reading the file from the 4th line and converts the lines into a list of strings. It then extracts the number of elements, element symbols, and other parameters from the list. Next, it reads the embedded energy and electron density data for each element, as well as the r*phi data for each interaction between elements. Finally, it sets up the ranges and arrays for the potential data and calls the `set_splines` function to calculate the splines.

Note: This function assumes that the potential data file is formatted correctly and contains the required information in the expected order.

"""
function read_potential!(calculator::EAM, fd::String)
    lines = readdlm(fd, '\n', String) # read the files, split by new line

    function lines_to_list(lines) # convert the entries in lines to list
        data = []
        for line in lines
            append!(data, split(line))
        end
        return data
    end

    i = 4 # start from the 4th line
    data = lines_to_list(lines[i:end])

    calculator.Nelements = parse(Int, data[1]) # number of elements
    d = 2
    calculator.elements = data[d:d+calculator.Nelements-1] # the elements symbols starts from the 2nd entries
    d += calculator.Nelements

    calculator.nrho = parse(Int, data[d]) 
    calculator.drho = parse(Float64, data[d+1])
    calculator.nr = parse(Int, data[d+2])
    calculator.dr = parse(Float64, data[d+3])# the cutoff radius in angstroms
    calculator.cutoff = parse(Float64, data[d+4]) 

    calculator.embedded_data = zeros(calculator.Nelements, calculator.nrho)
    calculator.density_data = zeros(calculator.Nelements, calculator.nr)
    calculator.Z = zeros(Int, calculator.Nelements)
    calculator.mass = zeros(calculator.Nelements)
    calculator.a = zeros(calculator.Nelements)
    calculator.lattice = ""
    d += 5

    # reads in the part of the eam file for each element
    for elem in 1:calculator.Nelements
        calculator.Z[elem] = parse(Int, data[d]) # the atomic number
        calculator.mass[elem] = parse(Float64, data[d+1]) # the atomic mass
        calculator.a[elem] = parse(Float64, data[d+2]) # the lattice constant
        calculator.lattice *= data[d+3] # the lattice type
        d += 4

        calculator.embedded_data[elem, :] = parse.(Float64, data[d:(d+calculator.nrho-1)]) # the embedded energy of the element
        d += calculator.nrho
        calculator.density_data[elem, :] = parse.(Float64, data[d:(d+calculator.nr-1)]) # the electron density of the element
        d += calculator.nr
    end

    # reads in the r*phi data for each interaction between elements
    calculator.rphi_data = zeros(calculator.Nelements, calculator.Nelements, calculator.nr)

    for i in 1:calculator.Nelements
        for j in 1:i
            calculator.rphi_data[j, i, :] = parse.(Float64, data[d:(d+calculator.nr-1)])
            d += calculator.nr
        end
    end

    calculator.r_range = (0:calculator.nr-1)*calculator.dr
    calculator.rho_range = (0:calculator.nrho-1)*calculator.drho
    calculator.r = collect(calculator.r_range)
    calculator.rho = collect(calculator.rho_range)

    set_splines(calculator)
end


read_potential!

### 1. Read potential

In [7]:
eam = EAM()
fname = "Al99.eam.alloy"
read_potential!(eam, fname)

### 2. Calculate potential energy

In [8]:
"""
    vector(c1, c2, boundary_side_lengths)

Displacement between two coordinate values from c1 to c2, accounting for
periodic boundary conditions.

The minimum image convention is used, so the displacement is to the closest
version of the coordinates accounting for the periodic boundaries.
For the [`TriclinicBoundary`](@ref) an approximation is used to find the closest
version by default.

vector_1D is from Molly.jl
"""
function Molly.vector(c1::SVector{3, Float64}, c2::SVector{3, Float64}, boundary_side_lengths::SVector{3, Float64})
    return @inbounds SVector(
        vector_1D(c1[1], c2[1], boundary_side_lengths[1]),
        vector_1D(c1[2], c2[2], boundary_side_lengths[2]),
        vector_1D(c1[3], c2[3], boundary_side_lengths[3]),
    )
end

function get_neighbors(neig, i)
    neighbors::Vector{Int} = []
    for j in 1:length(neig)
        neig_j = neig[j]
        if neig_j[1] == i
            append!(neighbors, neig_j[2])
        end
        if neig_j[2] == i
            append!(neighbors, neig_j[1])
        end
    end
    return unique(neighbors)
end

function get_neighbors_all(sys::Molly.System)
    neighbors_all = [Int[] for _ in 1:length(sys.atoms)]
    n_threads = 1
    neig = find_neighbors(sys, sys.neighbor_finder; n_threads=n_threads)
    for i in 1:length(neig)
        pair_i = neig[i]
        append!(neighbors_all[pair_i[1]], pair_i[2])
        append!(neighbors_all[pair_i[2]], pair_i[1])
    end
    return neighbors_all
end

function convert_neighbors(neig, n_atoms)
    neighbors_all = [Int[] for _ in 1:n_atoms]
    for i in 1:length(neig)
        pair_i = neig[i]
        append!(neighbors_all[pair_i[1]], pair_i[2])
        append!(neighbors_all[pair_i[2]], pair_i[1])
    end
    return neighbors_all
end

function get_type(index_list::Vector{Int64}, typelist::Vector{Int64})
    list_type_index = Vector{Int}(undef, length(index_list))
    for i in 1:length(index_list)
        list_type_index[i] = indexin(1, typelist)[1]
    end
    return list_type_index
end


get_type (generic function with 1 method)

In [9]:
"""
calculate_energy(eam::EAM, sys::Molly.System, neighbors_all)

Calculate the total energy of a system using the Embedded Atom Method (EAM).

# Arguments
- `eam::EAM`: The EAM calculator.
- `sys::Molly.System`: The system object containing atom coordinates and types.
- `neighbors_all`: A precomputed list of neighbors for each atom.

# Returns
- `energy::Float64`: The total energy of the system in electron volts (eV).
"""

function calculate_energy(eam::EAM, sys::Molly.System, neighbors_all::Vector{Vector{Int}})
    coords = [ustrip(coord) for coord in sys.coords]
    boundary = @SVector [(ustrip(sys.boundary[i])) for i in 1:3]
    return calculate_energy(eam, coords, boundary, neighbors_all)
end

function calculate_energy(eam::EAM, coords::Vector{SVector{3, Float64}}, boundary::SVector{3, Float64}, neighbors_all::Vector{Vector{Int}})    
    n_threads = 1

    pair_energy::Float64 = 0.0
    embedding_energy::Float64 = 0.0
    total_density::Vector{Float64} = zeros(length(coords))
    
    # for single element system, only one type is considered
    typelist = [1]
    i_type::Int = 1 in typelist ? indexin(1, typelist)[1] : error("1 not found in typelist")
    eam_phi_ix = eam.phi[i_type, :]

    for i::Int in 1:length(coords)
        # neighbors = get_neighbors(neig, i)
        neighbors::Vector{Int} = neighbors_all[i]
        coords_i = coords[i]
        
        if isempty(neighbors)
            continue
        end

        n_neighbors::Int = length(neighbors)
        d_i::Vector{Float64} = zeros(n_neighbors)
        for (index_j, j::Int) in enumerate(neighbors)
            d_i[index_j] = (sqrt(sum(vector(coords_i, coords[j], boundary).^2)))*10
        end

        eam_embedded_energy_i = eam.embedded_energy[i_type]
        for j_type::Int in 1:eam.Nelements
            # use = get_type(neighbors, typelist) .== j_type
            # if !any(use)
            #     continue
            # end
            # d_use = d_i[use]
            d_use = d_i

            pair_energy += sum(eam_phi_ix[j_type].(d_use))
            total_density[i] += sum(eam.electron_density[j_type].(d_use))
        end
        embedding_energy += eam_embedded_energy_i.(total_density[i])
    end

    components = Dict("pair" => pair_energy/2, "embedding" => embedding_energy)
    energy::Float64 = sum(values(components))
    return energy*u"eV"
end

calculate_energy (generic function with 2 methods)

In [10]:
atoms_ase_sim = convert_ase_custom(molly_system)

neighbors_all = get_neighbors_all(molly_system)

# run first time before timing
AtomsCalculators.potential_energy(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal)
calculate_energy(eam, molly_system, neighbors_all)

println("Calculating potential energy using ASE EAM calculator")
@time E_ASE = AtomsCalculators.potential_energy(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal)
println("Calculating potential energy using my EAM calculator")
@time E_my = calculate_energy(eam, molly_system, neighbors_all)
@printf("ASE EAM calculator: %e eV\n",ustrip(E_ASE))
@printf("My EAM calculator: %e eV\n",ustrip(E_my))
@printf("Difference: %e eV\n",ustrip(AtomsCalculators.potential_energy(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal) - calculate_energy(eam, molly_system, neighbors_all)))

function eam_ASE()
    AtomsCalculators.potential_energy(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal)
end
function eam_my()
    calculate_energy(eam, molly_system, neighbors_all)
end

n_repeat = 10
t0 = time()
E_ASE = repeat(eam_ASE, n_repeat)
t1 = time()
E_ASE = repeat(eam_my, n_repeat)
t2 = time()

println("time/atom/step by ASE EAM calculator: ", (t1-t0)/n_repeat/length(molly_system.atoms), " seconds")
println("time/atom/step by my EAM calculator: ", (t2-t1)/n_repeat/length(molly_system.atoms), " seconds")

Calculating potential energy using ASE EAM calculator
  0.028541 seconds (27.72 k allocations: 1.836 MiB)
Calculating potential energy using my EAM calculator
  0.000356 seconds (899 allocations: 379.031 KiB)
ASE EAM calculator: -9.424427e+02 eV
My EAM calculator: -9.424427e+02 eV
Difference: 0.000000e+00 eV
time/atom/step by ASE EAM calculator: 8.795867531986559e-6 seconds
time/atom/step by my EAM calculator: 7.217294078762249e-6 seconds


In [11]:
using ProfileView
ProfileView.@profview repeat(eam_my, n_repeat)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile  -  17:29:17", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip

### 3. Calculate force

In [12]:
"""
    calculate_forces(eam::EAM, sys::Molly.System, neighbors_all)

Calculate the forces on particles in a molecular system using the Embedded Atom Method (EAM).

# Arguments
- `eam::EAM`: An instance of the EAM potential.
- `sys::Molly.System`: The molecular system.
- `neighbors_all`: A matrix containing the neighbors of each particle in the system.

# Returns
- `forces_particle`: A matrix containing the forces on each particle in the system.
"""
function calculate_forces(eam::EAM, sys::Molly.System, neighbors_all::Vector{Vector{Int}})
    coords = [ustrip(coord) for coord in sys.coords]
    boundary = @SVector [ustrip(sys.boundary[i]) for i in 1:3]
    return calculate_forces(eam, coords, boundary, neighbors_all)
end

function calculate_forces(eam::EAM, coords::Vector{SVector{3, Float64}}, boundary::SVector{3, Float64}, neighbors_all::Vector{Vector{Int}})
    n_threads::Int = 1
    
    ## for single element system, only one type is considered
    typelist::Vector{Int} = [1]
    i_type::Int = 1 in typelist ? indexin(1, typelist)[1] : error("1 not found in typelist")
    d_electron_density_i = eam.d_electron_density[i_type]
    
    # preallocate
    # initialize forces_particle
    forces_particle::Matrix{Float64} = zeros(length(coords),3)
    
    # initialize total_density
    total_density::Vector{Float64} = zeros(length(coords))

    r_all::Vector{Matrix{Float64}} = []
    d_all::Vector{Vector{Float64}} = []

    n_neighbors_all::Vector{Int} = [length(neighbors_all[i]) for i in 1:length(coords)]
    n_neighbors_max::Int = maximum(n_neighbors_all)

    r_i::Matrix{Float64} = zeros(n_neighbors_max,3)
    d_i::Vector{Float64} = zeros(n_neighbors_max)
    for i::Int in 1:length(coords)
        # i_type::Int = indexin(1, typelist)[1]
        
        # neighbors = get_neighbors(neig, i)
        neighbors::Vector{Int} = neighbors_all[i]    
        if isempty(neighbors)
            continue
        end

        n_neighbors::Int = length(neighbors)
        coords_i = coords[i]
    
        # distance between atom i and its neighbors
        # r_i::Matrix{Float64} = zeros(n_neighbors,3)
        # d_i::Vector{Float64} = zeros(n_neighbors)
        for (index_j::Int, j::Int) in enumerate(neighbors)
            r_ij = (vector(coords_i, coords[j], boundary)*10) # convert to Å
            d_ij = sqrt(sum(r_ij.^2))
            r_i[index_j,1:3] = r_ij
            d_i[index_j] = d_ij
        end

        push!(r_all, r_i[1:n_neighbors,1:3])
        push!(d_all, d_i[1:n_neighbors])
    
        for j_type::Int in 1:eam.Nelements # iterate over all types
            # use = get_type(neighbors, typelist) .== j_type # get the index of the neighbors with type j
            # if !any(use)
            #     continue
            # end
            # d_use = d_i[use]
            d_use = d_all[i]
    
            density = sum(eam.electron_density[j_type].(d_use)) # electron density
            total_density[i] += density # total electron density around atom i
        end
    end
    
    # calculate forces on particles
    for i::Int in 1:length(coords)
        # i_type::Int = indexin(1, typelist)[1]
            
        # neighbors = get_neighbors(neig, i)
        neighbors::Vector{Int} = neighbors_all[i]
        if isempty(neighbors)
            continue
        end
        n_neighbors::Int = length(neighbors)
        coords_i = coords[i]
    
        r_i = r_all[i]
        d_i = d_all[i]
        
        # derivative of the embedded energy of atom i
        d_embedded_energy_i::Float64 = eam.d_embedded_energy[i_type].(total_density[i])
    
        ur_i = r_i
    
        # unit directional vector
        ur_i ./= d_i
        
        for j_type::Int in 1:eam.Nelements
            # use = get_type(neighbors, typelist) .== j_type # get the index of the neighbors with type j
            # if !any(use)
            #     continue
            # end
            # d_use = d_i[use]
            # ur_use = ur_i[use, :]
            # neighbors_use = neighbors[use]
            d_use = d_i
            ur_use::Matrix{Float64} = ur_i[:,:]
            neighbors_use = neighbors
    
            total_density_j = total_density[neighbors_use]
            
            scale::Vector{Float64} = eam.d_phi[i_type, j_type].(d_use)
            scale .+= (d_embedded_energy_i .* eam.d_electron_density[j_type].(d_use)) 
            scale .+= (eam.d_embedded_energy[j_type].(total_density_j) .* d_electron_density_i.(d_use))
    
            forces_particle[i, :] .+= (ur_use' * scale)
        end
    end

    return [SVector{3,Float64}(forces_particle[idx_f,:]) for idx_f in 1:size(forces_particle)[1]]*u"eV/Å"
end

calculate_forces (generic function with 2 methods)

In [13]:
# run first time before timing
forces_ASE = AtomsCalculators.forces(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal)
forces_my = calculate_forces(eam, molly_system, neighbors_all)

println("Calculating forces using ASE EAM calculator")
@time forces_ASE = AtomsCalculators.forces(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal)
println("Calculating forces using My EAM calculator")
@time forces_my = calculate_forces(eam, molly_system, neighbors_all)

@printf("Sum of forces by ASE EAM calculator: [%e %e %e] eV/Å\n",ustrip(sum(forces_ASE))...)
@printf("Sum of forces by my EAM calculator: [%e %e %e] eV/Å\n",ustrip(sum(forces_my))...)

forces_err = forces_my - forces_ASE
index_max_forces_err = argmax([sqrt(sum(fe.^2)) for fe in forces_err])
@printf("Max force error: %e eV/Å\n", ustrip(sqrt(sum(forces_err[index_max_forces_err].^2))))

function eam_ASE_f()
    AtomsCalculators.forces(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal)
end
function eam_my_f()
    calculate_forces(eam, molly_system, neighbors_all)
end

eam_ASE_f()
eam_my_f()
n_repeat = 10
t0 = time()
E_ASE = repeat(eam_ASE_f, n_repeat)
t1 = time()
E_ASE = repeat(eam_my_f, n_repeat)
t2 = time()

println("time/atom/step by ASE EAM calculator: ", (t1-t0)/n_repeat/length(molly_system.atoms), " seconds")
println("time/atom/step by my EAM calculator: ", (t2-t1)/n_repeat/length(molly_system.atoms), " seconds")


Calculating forces using ASE EAM calculator
  0.005702 seconds (27.76 k allocations: 1.850 MiB)
Calculating forces using My EAM calculator
  0.000946 seconds (2.68 k allocations: 1.238 MiB)
Sum of forces by ASE EAM calculator: [-4.529710e-14 -4.629630e-14 -2.220446e-15] eV/Å
Sum of forces by my EAM calculator: [1.887379e-15 -4.440892e-16 -3.996803e-15] eV/Å
Max force error: 1.862710e-06 eV/Å
time/atom/step by ASE EAM calculator: 9.392479718741724e-6 seconds
time/atom/step by my EAM calculator: 4.952883316298663e-6 seconds


In [14]:
using ProfileView
ProfileView.@profview repeat(eam_my_f, n_repeat)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile  -  17:29:22", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip

## Incoporate into `AtomCalculators` force/energy calculator and `Molly` simulator

### Define customized interaction type in AtomsCalculators

In [15]:
struct EAMInteractionJulia
    calculator::Any  # Holds the ASE EAM calculator reference
    f_energy::Any    # Holds the energy function
    f_forces::Any    # Holds the forces function
end

# function AtomsCalculators.potential_energy(system::Molly.System, neighbors)
#     n_atoms = length(system.atoms)
#     neighbors_all = convert_neighbors(neighbors,n_atoms)
#     interaction = system.general_inters[1]
#     return AtomsCalculators.potential_energy(system, interaction, neighbors_all)
# end

function AtomsCalculators.potential_energy(system::Molly.System, interaction::EAMInteractionJulia, neighbors_all::Vector{Vector{Int}}; kwargs...)    
    # Calculate potential energy
    energy = interaction.f_energy(interaction.calculator, system, neighbors_all)
    return energy
end

# function AtomsCalculators.forces(system::Molly.System, neighbors)
#     n_atoms = length(system.atoms)
#     neighbors_all = convert_neighbors(neighbors,n_atoms)
#     interaction = system.general_inters[1]
#     return AtomsCalculators.forces(system, interaction, neighbors_all)
# end

function AtomsCalculators.forces(system::Molly.System, interaction::EAMInteractionJulia, neighbors_all::Vector{Vector{Int}}; kwargs...)
    # Calculate forces
    energy = interaction.f_forces(interaction.calculator, system, neighbors_all)
    return energy
end

In [16]:
function initialize_system(loggers=(coords=CoordinateLogger(1),))
    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms,
    atoms_data = [AtomData(element="Al") for a in molly_atoms],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DistanceNeighborFinder(
    eligible=trues(length(molly_atoms), length(molly_atoms)),
    n_steps=10,
    dist_cutoff=6.3/10*u"nm"),
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/nm",  # Ensure these units are correctly specified
    general_inters=[eam_Julia])
    return system_init
end

initialize_system (generic function with 2 methods)

### Define ABCSimulator

In [17]:
# Define the ABCSimulator structure
"""
In the constructor function ABCSimulator, default values are provided for each of these fields. 
If you create a SteepestDescentMinimizer without specifying the types, default values 
will determine the types of the fields. For example, if you create a ABCSimulator without specifying sigma, 
it will default to 0.01*u"nm", and S will be the type of this value.
"""
struct ABCSimulator{S,W,D,F,L}
    sigma::S 
    W::W
    max_steps::Int
    max_steps_minimize::Int
    step_size_minimize::D
    tol::F
    log_stream::L
end

"""
ABCSimulator(; sigma=0.01*u"nm", W=1e-2*u"eV", max_steps=100, max_steps_minimize=100, step_size_minimize=0.01u"nm", tol=1e-10u"kg*m*s^-2", log_stream=devnull)

Constructor for ABCSimulator.

## Arguments
- `sigma`: The value of sigma in units of nm.
- `W`: The value of W in units of eV.
- `max_steps`: The maximum number of steps for the simulator.
- `max_steps_minimize`: The maximum number of steps for the minimizer.
- `step_size_minimize`: The step size for the minimizer in units of nm.
- `tol`: The tolerance for convergence in units of kg*m*s^-2.
- `log_stream`: The stream to log the output.

## Returns
- An instance of ABCSimulator.
"""
function ABCSimulator(;
                        sigma=0.01*u"nm", W=1e-2*u"eV", max_steps=100, max_steps_minimize=100, step_size_minimize=0.01u"nm",tol=1e-10u"kg*m*s^-2",
                        log_stream=devnull)
    return ABCSimulator(sigma, W, max_steps, max_steps_minimize, step_size_minimize, tol, log_stream)
end

# Penalty function with Gaussuan form
"""
Returns a penalty function of system coordinate x with Gaussuan form
x:      System coordinate
x_0:    Reference system coordinate
sigma:  Spatial extent of the activation, per sqrt(degree of freedom)
W:      Strenth of activation, per degree of freedom
pbc:    Periodic boundary conditions
"""
function f_phi_p(x, x_0, sigma, W, pbc=nothing)
    N = length(x)
    
    sigma2_new = sigma^2*3*N
    EDSQ = (A, B) -> sum(sum(map(x -> x.^2, A-B)))
    phi_p = sum([W * exp(-EDSQ(x,c) / (2*sigma2_new)) for c in x_0])
    return phi_p
end

function grad_f_phi_p(x, x_0, sigma, W)
    N = length(x)
    sigma2_new = sigma^2*3*N
    EDSQ = (A, B) -> sum(sum(map(x -> x.^2, A-B)))
    grad_phi_p = sum([W * exp(-EDSQ(x,c) / (2*sigma2_new)) * (x - c) / sigma2_new for c in x_0])
    return grad_phi_p
end

# Calculate the gradient of the penalty energy
function penalty_forces(sys, penalty_coords, sigma, W)
    # Function of the penalty energy for a given coordinate
    f_phi_p_coords = x -> f_phi_p(x, penalty_coords, sigma, W)

    # Calculate the gradient of the penalty energy, The penalty force is the negative gradient of the penalty energy
    penalty_fs = -gradient(f_phi_p_coords, sys.coords)[1]
    # penalty_fs = -grad_f_phi_p(sys.coords, penalty_coords, sigma, W)

    return penalty_fs
end

# Define the forces function with penalty term
"""
Evaluate the forces acting on the system with penalty term
If there is no penalty term, the penalty_coords should be set to nothing, 
and return the forces identical to the original forces function
"""
function Molly.forces(sys::System, interaction, penalty_coords, sigma, W, neighbors_all;
    n_threads::Integer=Threads.nthreads()) 

    
    fs = calculate_forces(eam, molly_system, neighbors_all)

    # Add penalty term to forces
    if penalty_coords != nothing
        fs += penalty_forces(sys, penalty_coords, sigma, W)
    end

    return fs
end

# Define the Minimize! function, on the basis of the Molly SteepestDescentMinimizer
"""
Minimize!(sys, sim, penalty_coords; n_threads::Integer, frozen_atoms=[])

Minimizes the system `sys` energy using the simulatior `sim` providing penalty coordinates `penalty_coords`.

# Arguments
- `sys`: The system to be minimized.
- `sim`: The simulation object.
- `penalty_coords`: The penalty coordinates used in the minimization.
- `n_threads`: The number of threads to use in the minimization.
- `frozen_atoms`: (optional) A list of atoms to be frozen during the minimization.

# Examples
"""
function Minimize!(sys, sim, interaction, penalty_coords, neighbors_all; n_threads::Integer, frozen_atoms=[])
    using_constraints = length(sys.constraints) > 0
    hn = sim.step_size_minimize
    E_phi = 0*u"eV"
    if penalty_coords!=nothing
        E_phi += f_phi_p(sys.coords, penalty_coords, sim.sigma, sim.W)
    end
    E = calculate_energy(interaction, molly_system, neighbors_all) + E_phi
    
    # Set F_multiplier of frozen_atoms to zero
    F_multiplier = ones(length(sys.coords))

    for atom in frozen_atoms
        F_multiplier[atom] = 0
    end

    for step_n in 1:sim.max_steps_minimize
        # Calculate the forces using the new forces function
        # penalty_coords is fixed throughout the minimization
        F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all; n_threads=1)
        F = F.*F_multiplier
        max_force = maximum(norm.(F))
        
        coords_copy = sys.coords
        sys.coords += hn * F ./ max_force
        using_constraints && apply_position_constraints!(sys, coords_copy; n_threads=n_threads)

        neighbors_all_copy = neighbors_all
        
        # System energy after applying displacements in this step
        E_phi = 0*u"eV"
        if penalty_coords!=nothing
            E_phi += f_phi_p(sys.coords, penalty_coords, sim.sigma, sim.W)
        end

        E_trial = calculate_energy(interaction, molly_system, neighbors_all) + E_phi
        if E_trial < E
            hn = 6 * hn / 5
            E = E_trial
            # println(sim.log_stream, "Step ", step_n, " - potential energy ",
            #         E_trial, " - max force ", max_force, " - accepted")
        else
            sys.coords = coords_copy
            neighbors_all = neighbors_all_copy
            hn = hn / 5
            # println(sim.log_stream, "Step ", step_n, " - potential energy ",
            #         E_trial, " - max force ", max_force, " - rejected")
        end

        if max_force < sim.tol
            break
        end
    end
    F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all; n_threads=1)
    F = F.*F_multiplier
    max_force = maximum(norm.(F))
    @printf("max force = %e N ",ustrip(max_force))
    return sys
end

# Implement the simulate! function for ABCSimulator
"""
simulate!(sys, sim::ABCSimulator; n_threads::Integer=Threads.nthreads(), frozen_atoms=[], run_loggers=true, fname="output.txt")

Simulates the system using the ABCSimulator.

# Arguments
- `sys`: The system to be simulated.
- `sim`: An instance of the ABCSimulator.
- `n_threads`: The number of threads to use for parallel execution. Defaults to the number of available threads.
- `frozen_atoms`: A list of atoms that should be frozen during the simulation.
- `run_loggers`: A boolean indicating whether to run the loggers during the simulation.
- `fname`: The name of the output file.

# Examples 
simulate!(molly_system, simulator, n_threads=1, fname="output_test.txt", frozen_atoms=frozen_atoms)
"""
function simulate!(sys, sim::ABCSimulator, interaction::EAM; n_threads::Integer=Threads.nthreads(), frozen_atoms=[], run_loggers=true, fname="output.txt", neig_inteval=1, minimize_only=false)

    # Do not wrap the coordinates
    neighbors_all = get_neighbors_all(sys)
    neighbors = find_neighbors(sys, sys.neighbor_finder; n_threads=n_threads)

    # open an empty output file
    open(fname, "w") do file
        write(file, "")
    end

    # Set d_multiplier of frozen_atoms to zero
    d_multiplier = ones(length(sys.coords))
    for atom in frozen_atoms
        d_multiplier[atom] = 0
    end

    # 0. Call Minimize! without penalty_coords before the loop
    Minimize!(sys, sim, interaction, nothing, neighbors_all; n_threads=n_threads, frozen_atoms=frozen_atoms)
    E = calculate_energy(interaction, molly_system, neighbors_all)

    # Run the loggers, Log the step number (or other details as needed)
    run_loggers!(sys, neighbors, 0, run_loggers; n_threads=n_threads)
    @printf("step %d: ",0)
    print(E)
    print("\n")

    if minimize_only
        return sys
    end

    # 1. Store the initial coordinates
    penalty_coords = [copy(sys.coords)]  

    for step_n in 1:sim.max_steps
        if step_n % neig_inteval == 0
            neighbors_all = get_neighbors_all(sys)
        end

        # 2. Slightly perturb the system coordinates
        for i in 1:length(sys.coords)
            random_direction = randn(size(sys.coords[i]))
            sys.coords[i] += 1e-4*u"nm" * random_direction*d_multiplier[i]
        end

        # 3. Call Minimize! with penalty_coords, update system coordinates
        Minimize!(sys, sim, interaction, penalty_coords, neighbors_all; n_threads=n_threads, frozen_atoms=frozen_atoms)
        E = calculate_energy(interaction, molly_system, neighbors_all)
        @printf("step %d: ",step_n)
        print(E)
        print("\n")

        # Run the loggers, Log the step number (or other details as needed)
        run_loggers!(sys, neighbors, step_n, run_loggers; n_threads=n_threads)
        println(sim.log_stream, "Step 0 - potential energy ",
                E, " - max force N/A - N/A")
        E_phi = f_phi_p(sys.coords, penalty_coords, sim.sigma, sim.W)
        open(fname, "a") do file
            write(file, string(ustrip(E))*" "*string(ustrip(E_phi))*"\n")
        end

        # 4. Update penalty_coords for the next step
        push!(penalty_coords, copy(sys.coords))
    end
    return sys
end

simulate!

### Initialize System and run simulator

In [18]:
function initialize_system(loggers=(coords=CoordinateLogger(1),))
    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms,
    atoms_data = [AtomData(element="Al") for a in molly_atoms],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DistanceNeighborFinder(
    eligible=trues(length(molly_atoms), length(molly_atoms)),
    n_steps=10,
    dist_cutoff=5/10*u"nm"),
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/nm"  # Ensure these units are correctly specified
    )
    return system_init
end

initialize_system (generic function with 2 methods)

In [19]:
molly_system = initialize_system()

z_coords = [coords[3] for coords in molly_system.coords]
frozen_atoms = [index for (index, z_coord) in enumerate(z_coords) if z_coord < al_LatConst*5*0.1*u"nm"]
print(length(frozen_atoms))
print("\n")

98


In [20]:
sigma = 2e-3
W = 0.1
@printf("sigma = %e nm/dof^1/2\n W = %e eV",ustrip(sigma),ustrip(W))

simulator = ABCSimulator(sigma=sigma*u"nm", W=W*u"eV", max_steps=10, max_steps_minimize=500, step_size_minimize=1.5e-3u"nm", tol=1e-15u"kg*m*s^-2")
@time simulate!(molly_system, simulator, eam, n_threads=1, fname="output_test_Julia.txt", frozen_atoms=frozen_atoms, neig_inteval=10, minimize_only=false)

sigma = 2.000000e-03 nm/dof^1/2
 W = 1.000000e-01 eVmax force = 5.376086e-07 N step 0: -947.9384660262092 eV
max force = 1.033882e-15 N step 1: -947.9047975431001 eV
max force = 3.530783e-12 N step 2: -947.8930186140122 eV
max force = 8.043572e-15 N step 3: -947.8750142224677 eV
max force = 1.834068e-15 N step 4: -947.8413568836082 eV
max force = 3.057424e-14 N step 5: -947.8403206698395 eV
max force = 4.706763e-14 N step 6: -947.8587729920421 eV
max force = 1.862565e-14 N step 7: -947.8202294709075 eV
max force = 1.050296e-11 N step 8: -947.8335504077122 eV
max force = 3.671313e-12 N step 9: -947.8292037565662 eV
max force = 1.012330e-12 N step 10: -947.7609433352853 eV
113.255336 seconds (926.15 M allocations: 45.212 GiB, 4.54% gc time, 5.38% compilation time)


System with 295 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[2.0046477246638625 nm, 2.0046477246638625 nm, 4.2521 nm])